In [ ]:
import requests
import pandas as pd
import time

# Authentication for GitHub API
headers = {'Authorization': 'token -'}


In [ ]:
# Helper function to clean up company names
def clean_company_name(company):
    if company:
        company = company.strip().lstrip('@').upper()
    return company


In [ ]:
def fetch_users(city="Barcelona", min_followers=100):
    users = []
    page = 1

    while True:
        url = f"https://api.github.com/search/users?q=location:{city}+followers:>{min_followers}&page={page}&per_page=100"
        response = requests.get(url, headers=headers)
        data = response.json()

        # Break if no more results
        if 'items' not in data or not data['items']:
            break

        for user in data['items']:
            # Get full user info
            user_url = user['url']
            user_response = requests.get(user_url, headers=headers)
            user_data = user_response.json()

            # Extract required fields
            users.append({
                'login': user_data['login'],
                'name': user_data['name'],
                'company': clean_company_name(user_data['company']),
                'location': user_data['location'],
                'email': user_data['email'],
                'hireable': user_data['hireable'],
                'bio': user_data['bio'],
                'public_repos': user_data['public_repos'],
                'followers': user_data['followers'],
                'following': user_data['following'],
                'created_at': user_data['created_at'],
            })
        page += 1
        time.sleep(1)  # Avoid hitting API rate limits

    return users

In [ ]:
users = fetch_users()
users_df = pd.DataFrame(users)
users_df.to_csv('users.csv', index=False)

In [ ]:
# Function to fetch repositories for a user
def fetch_repositories(user_login):
    repositories = []
    page = 1

    while True:
        url = f"https://api.github.com/users/{user_login}/repos?per_page=100&page={page}"
        response = requests.get(url, headers=headers)
        repo_data = response.json()

        # Break if no more repositories
        if not repo_data:
            break

        for repo in repo_data:
            repositories.append({
                'login': user_login,
                'full_name': repo['full_name'],
                'created_at': repo['created_at'],
                'stargazers_count': repo['stargazers_count'],
                'watchers_count': repo['watchers_count'],
                'language': repo['language'],
                'has_projects': repo['has_projects'],
                'has_wiki': repo['has_wiki'],
                'license_name': repo['license']['key'] if repo['license'] else None,
            })

        # If fewer than 100 repositories are returned, it means we're on the last page
        if len(repo_data) < 100:
            break

        page += 1  # Move to the next page
        time.sleep(1)  # Avoid hitting API rate limits

    return repositories




In [ ]:
all_repositories = []
for user in users:
    user_repos = fetch_repositories(user["login"])
    all_repositories.extend(user_repos)

Fetched 194 repositories for user midudev
Fetched 85 repositories for user ai
Fetched 26 repositories for user raysan5
Fetched 451 repositories for user vfarcic
Fetched 135 repositories for user spite
Fetched 15 repositories for user LeCoupa
Fetched 72 repositories for user peduarte
Fetched 4 repositories for user jk4freedom
Fetched 127 repositories for user facontidavide
Fetched 22 repositories for user carloscuesta
Fetched 191 repositories for user gaborcsardi
Fetched 18 repositories for user amix
Fetched 35 repositories for user fxn
Fetched 14 repositories for user iturbides
Fetched 5 repositories for user rgomezcasas
Fetched 9 repositories for user cfenollosa
Fetched 24 repositories for user csorlandi
Fetched 349 repositories for user wlsf82
Fetched 5 repositories for user alexfarrerodev
Fetched 463 repositories for user ajsb85
Fetched 56 repositories for user carlosbuenosvinos
Fetched 67 repositories for user mariodev12
Fetched 200 repositories for user chicoxyzzy
Fetched 27 repos

In [ ]:
repos_df = pd.DataFrame(all_repositories)
repos_df.to_csv('repositories.csv', index=False)

1. Who are the top 5 users in Barcelona with the highest number of followers? List their login in order, comma-separated.

In [ ]:
import pandas as pd

# Load the users dataset
users_df = pd.read_csv('users.csv')

# Sort by 'followers' in descending order and select the top 5
top_5_users = users_df.sort_values(by='followers', ascending=False).head(5)

# Get the 'login' values of the top 5 users and join them in a comma-separated format
top_5_logins = ','.join(top_5_users['login'])

top_5_logins


'midudev,ai,raysan5,vfarcic,spite'

2. Who are the 5 earliest registered GitHub users in Barcelona? List their login in ascending order of created_at, comma-separated.

In [ ]:
users_df['created_at'] = pd.to_datetime(users_df['created_at'])

# Sort by 'created_at' in ascending order and select the top 5
earliest_users = users_df.sort_values(by='created_at', ascending=True).head(5)

# Get the 'login' values of the 5 earliest users and join them in a comma-separated format
earliest_logins = ','.join(earliest_users['login'])

earliest_logins

'oleganza,gravityblast,fesplugas,fxn,pauek'

3. What are the 3 most popular license among these users? Ignore missing licenses. List the license_name in order, comma-separated.

In [ ]:
import pandas as pd

# Load the repositories dataset
repos_df = pd.read_csv('repositories.csv')

repos_df.head()

,login,full_name,created_at,stargazers_count,watchers_count,language,has_projects,has_wiki,license_name
0,midudev,midudev/accordion-menu,2018-10-08T13:41:57Z,2,2,JavaScript,True,True,NaN
1,midudev,midudev/actas-2024,2024-08-01T15:04:40Z,42,42,NaN,True,True,NaN
2,midudev,midudev/adventjs-issues,2021-12-01T15:27:21Z,33,33,NaN,True,True,NaN
3,midudev,midudev/algolia-pwa-ecommerce,2022-01-03T20:17:22Z,24,24,TypeScript,True,True,mit
4,midudev,midudev/ant-design,2018-09-13T15:08:22Z,2,2,TypeScript,True,True,mit


In [ ]:
# Filter out rows with missing licenses
repos_df = repos_df[repos_df['license_name'].notna()]

# Count the occurrences of each license and get the top 3
top_licenses = repos_df['license_name'].value_counts().head(3)

# Get the names of the top 3 licenses and join them in a comma-separated format
top_licenses_list = ','.join(top_licenses.index)

top_licenses_list


'mit,apache-2.0,other'

4. Which company do the majority of these developers work at?

In [ ]:
non_empty_companies = users_df[users_df['company'].notna() & (users_df['company'] != '')]

# Find the most frequently occurring company
top_company = non_empty_companies['company'].value_counts().idxmax()

top_company

'FREELANCE'

5. Which programming language is most popular among these users?

In [ ]:
non_empty_languages = repos_df[repos_df['language'].notna()]

# Find the most frequently occurring language
top_language = non_empty_languages['language'].value_counts().idxmax()

top_language

'JavaScript'

6. Which programming language is the most popular among users who joined after 2020?

In [7]:
import pandas as pd

# Load the users and repositories datasets
users_df = pd.read_csv('users.csv')
repos_df = pd.read_csv('repositories.csv')

# Convert 'created_at' in users_df to datetime format
users_df['created_at'] = pd.to_datetime(users_df['created_at'])

# Filter users who joined after 2020
recent_users = users_df[users_df['created_at'] > '2020-12-31']

# Get the logins of these recent users
recent_user_logins = recent_users['login'].tolist()

# Filter repositories belonging to these recent users
recent_user_repos = repos_df[repos_df['login'].isin(recent_user_logins)]

# Filter out rows with missing language information
non_empty_languages = recent_user_repos[recent_user_repos['language'].notna()]

# Find the most popular language after 2020
top_languages = non_empty_languages['language'].value_counts().idxmax()

top_languages

'Python'

7. Which language has the highest average number of stars per repository?


In [9]:
import pandas as pd

# Load the repositories dataset
repos_df = pd.read_csv('repositories.csv')

# Filter out rows with missing language information
non_empty_languages = repos_df[repos_df['language'].notna()]

# Group by language and calculate the average stars per repository
avg_stars_per_language = non_empty_languages.groupby('language')['stargazers_count'].mean()

# Find the language with the highest average stars
top_language_by_stars = avg_stars_per_language.idxmax()
highest_avg_stars = avg_stars_per_language.max()

top_language_by_stars, highest_avg_stars


('Vim Script', 3839.0)

8. Let's define leader_strength as followers / (1 + following). Who are the top 5 in terms of leader_strength? List their login in order, comma-separated.

In [11]:
import pandas as pd

# Load the users dataset
users_df = pd.read_csv('users.csv')

# Calculate leader_strength for each user
users_df['leader_strength'] = users_df['followers'] / (1 + users_df['following'])

# Sort by leader_strength in descending order and select the top 5 users
top_5_leaders = users_df.sort_values(by='leader_strength', ascending=False).head(5)

# Get the 'login' values of the top 5 users and join them in a comma-separated format
top_5_logins = ','.join(top_5_leaders['login'])

top_5_logins


'midudev,vfarcic,spite,amix,cfenollosa'

9. What is the correlation between the number of followers and the number of public repositories among users in Barcelona?

In [12]:
import pandas as pd

# Load the users dataset
users_df = pd.read_csv('users.csv')

# Calculate the correlation between 'followers' and 'public_repos'
correlation = users_df['followers'].corr(users_df['public_repos'])

correlation


0.07123268320455914

10. Does creating more repos help users get more followers? Using regression, estimate how many additional followers a user gets per additional public repository.

In [15]:
import pandas as pd
from sklearn.linear_model import LinearRegression

# Load the users dataset
users_df = pd.read_csv('users.csv')

# Reshape the data for sklearn
X = users_df[['public_repos']]
y = users_df['followers']

# Perform linear regression
model = LinearRegression()
model.fit(X, y)

# The coefficient (slope) of the regression line gives the estimated followers gained per additional repository
followers_per_repo = model.coef_[0]

followers_per_repo


1.0308623319576913

11. Do people typically enable projects and wikis together? What is the correlation between a repo having projects enabled and having wiki enabled?

In [78]:
import pandas as pd

# Load the repositories dataset
repos_df = pd.read_csv('repositories.csv')


# Calculate the correlation to 3 decimal places
correlation = repos_df['has_projects'].corr(repos_df['has_wiki']).round(3)

correlation


0.317

12. Do hireable users follow more people than those who are not hireable?

In [76]:
# Load the users dataset
users_df = pd.read_csv('users.csv')

# Drop rows with NaN values in 'hireable' or 'following'
users_df = users_df.dropna(subset=['hireable', 'following'])

# Convert 'hireable' to boolean values
users_df['hireable'] = users_df['hireable'].astype(bool)

# Calculate the average 'following' for hireable and non-hireable users
avg_following_hireable = users_df[users_df['hireable']]['following'].mean()
avg_following_non_hireable = users_df[~users_df['hireable']]['following'].mean()

# Calculate the difference and round to 3 decimal places
following_difference = round(avg_following_hireable - avg_following_non_hireable, 3)

following_difference



nan

13. Some developers write long bios. Does that help them get more followers? What's the impact of the length of their bio (in Unicode words, split by whitespace) with followers?

In [30]:
import pandas as pd
from sklearn.linear_model import LinearRegression

# Load the users dataset
users_df = pd.read_csv('users.csv')

# Filter out rows where 'bio' is missing
users_with_bio = users_df.dropna(subset=['bio'])

# Calculate bio word count by splitting on whitespace and counting words
users_with_bio['bio_word_count'] = users_with_bio['bio'].apply(lambda x: len(x.split()))

# Define independent (X) and dependent (y) variables
X = users_with_bio[['bio_word_count']]  # Bio word count
y = users_with_bio['followers']         # Number of followers

# Perform linear regression
model = LinearRegression()
model.fit(X, y)

# Get the regression slope (coefficient for bio word count)
slope = model.coef_[0]

slope


<ipython-input-30-803fb95b1df2>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  users_with_bio['bio_word_count'] = users_with_bio['bio'].apply(lambda x: len(x.split()))


13.72050854380058

14. Who created the most repositories on weekends (UTC)? List the top 5 users' login in order, comma-separated

In [31]:
import pandas as pd

# Load the repositories dataset
repos_df = pd.read_csv('repositories.csv')

# Convert 'created_at' to datetime format
repos_df['created_at'] = pd.to_datetime(repos_df['created_at'], utc=True)

# Filter for repositories created on weekends (Saturday and Sunday)
repos_df['day_of_week'] = repos_df['created_at'].dt.dayofweek
weekend_repos = repos_df[repos_df['day_of_week'] >= 5]  # 5 = Saturday, 6 = Sunday

# Count the number of repositories created by each user on weekends
weekend_repo_counts = weekend_repos['login'].value_counts().head(5)

# Get the top 5 users' logins in a comma-separated format
top_5_weekend_creators = ','.join(weekend_repo_counts.index)

top_5_weekend_creators


'nilportugues,kinow,ajsb85,vfarcic,wlsf82'

15. Do people who are hireable share their email addresses more often?

In [40]:
# Drop rows with NaN values in 'hireable' column
users_df = users_df.dropna(subset=['hireable'])

# Convert 'hireable' to boolean values
users_df['hireable'] = users_df['hireable'].astype(bool)

# Calculate the fraction of hireable and non-hireable users with an email
fraction_hireable_with_email = users_df[users_df['hireable']]['email'].notna().mean()
fraction_non_hireable_with_email = users_df[~users_df['hireable']]['email'].notna().mean()

# Calculate the difference and round to 3 decimal places
email_share_difference = round(fraction_hireable_with_email - fraction_non_hireable_with_email, 3)

email_share_difference



nan

16. Let's assume that the last word in a user's name is their surname (ignore missing names, trim and split by whitespace.) What's the most common surname? (If there's a tie, list them all, comma-separated, alphabetically)

In [36]:
import pandas as pd

# Load the users dataset
users_df = pd.read_csv('users.csv')

# Filter out rows with missing names
users_with_names = users_df.dropna(subset=['name'])

# Extract the last word as surname (trim and split by whitespace)
users_with_names['surname'] = users_with_names['name'].str.strip().apply(lambda x: x.split()[-1])

# Count the occurrences of each surname
surname_counts = users_with_names['surname'].value_counts()

# Find the highest frequency
max_count = surname_counts.max()

# Get all surnames with the highest frequency, sort alphabetically, and join with commas
most_common_surnames = ','.join(sorted(surname_counts[surname_counts == max_count].index))

most_common_surnames


<ipython-input-36-da8fa9a89dc2>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  users_with_names['surname'] = users_with_names['name'].str.strip().apply(lambda x: x.split()[-1])


'Martínez,Ortiz'